# Parte 1 - Analisando os dados

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
meteor_showers = pd.read_csv('data/meteorshowers.csv')
moon_phases = pd.read_csv('data/moonphases.csv')
constellations = pd.read_csv('data/constellations.csv')
cities = pd.read_csv('data/cities.csv')

In [3]:
change_meteor_shower = {'name':'Chang\'e','radiant':'Draco','bestmonth':'october','startmonth':'october','startday':1,'endmonth':'october','endday':31,'hemisphere':'northern','preferredhemisphere':'northern'}

meteor_showers = pd.concat([meteor_showers, pd.DataFrame(change_meteor_shower, index=[0])], ignore_index=True)

In [4]:
draco_constellation = {'constellation':'Draco','bestmonth':'july','latitudestart':90,'latitudeend':-15,'besttime':2100,'hemisphere':'northern'}

constellations = pd.concat([constellations, pd.DataFrame(draco_constellation, index=[0])], ignore_index=True)

In [5]:
meteor_showers.head()
meteor_showers.info()
moon_phases.head()
moon_phases.info()
constellations.head()
constellations.info()
cities.head()
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name                 6 non-null      object
 1   radiant              6 non-null      object
 2   bestmonth            6 non-null      object
 3   startmonth           6 non-null      object
 4   startday             6 non-null      int64 
 5   endmonth             6 non-null      object
 6   endday               6 non-null      int64 
 7   hemisphere           6 non-null      object
 8   preferredhemisphere  6 non-null      object
dtypes: int64(2), object(7)
memory usage: 560.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   month         366 non-null    object
 1   day           366 non-null    int64 
 2   moonphase     50 non-null    

Criando um mapa de meses para números e mapeando os meses para as colunas que possuem meses:

In [6]:
months = {'january':1, 'february':2, 'march':3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 'september':9, 'october':10, 'november':11, 'december':12}
meteor_showers.bestmonth = meteor_showers.bestmonth.map(months)
meteor_showers.startmonth = meteor_showers.startmonth.map(months)
meteor_showers.endmonth = meteor_showers.endmonth.map(months)
moon_phases.month = moon_phases.month.map(months)
constellations.bestmonth = constellations.bestmonth.map(months)

Convertendo meses e dias de meteor_showers e moon_phases em um tipo chamado "datetime":

In [7]:
meteor_showers['startdate'] = pd.to_datetime(2020*10000+meteor_showers.startmonth*100+meteor_showers.startday,format='%Y%m%d')
meteor_showers['enddate'] = pd.to_datetime(2020*10000+meteor_showers.endmonth*100+meteor_showers.endday,format='%Y%m%d')
moon_phases['date'] = pd.to_datetime(2020*10000+moon_phases.month*100+moon_phases.day,format='%Y%m%d')

Convertendo os dados do hemisfério em números utilizando o processo de mapeamento:

In [8]:
hemispheres = {'northern':0, 'southern':1, 'northern, southern':3}
meteor_showers.hemisphere = meteor_showers.hemisphere.map(hemispheres)
constellations.hemisphere = constellations.hemisphere.map(hemispheres)

Convertendo as fases da Lua em números que representam o percentual da Lua que está visível, adicionando uma nova coluna para representar os dados:

In [9]:
phases = {'new moon':0,'third quarter':0.5, 'first quarter':0.5,'full moon':1.0}
moon_phases['percentage'] = moon_phases.moonphase.map(phases)
moon_phases.head()

,month,day,moonphase,specialevent,date,percentage
0,1,1,NaN,NaN,2020-01-01,NaN
1,1,2,first quarter,NaN,2020-01-02,0.5
2,1,3,NaN,NaN,2020-01-03,NaN
3,1,4,NaN,NaN,2020-01-04,NaN
4,1,5,NaN,NaN,2020-01-05,NaN


Removendo colunas inúteis:

In [10]:
meteor_showers = meteor_showers.drop(['startmonth', 'startday', 'endmonth', 'endday', 'hemisphere'], axis=1)
moon_phases = moon_phases.drop(['month','day','moonphase','specialevent'], axis=1)
constellations = constellations.drop(['besttime'], axis=1)

Executando um loop em cada linha e coluna no DataFrame moon_phases, substituindo o valor NaN pela última fase que apareceu que está salva na variável criada:


In [11]:
lastPhase = 0

for index, row in moon_phases.iterrows():
    if pd.isnull(row['percentage']):
        moon_phases.at[index,'percentage'] = lastPhase
    else:
        lastPhase = row['percentage']

moon_phases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        366 non-null    datetime64[ns]
 1   percentage  366 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB


# Parte 2 - Escrevendo uma Função de Previsão

Examinando os conjuntos de dados:

In [12]:
meteor_showers.info()
moon_phases.info()
constellations.info()
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   name                 6 non-null      object        
 1   radiant              6 non-null      object        
 2   bestmonth            6 non-null      int64         
 3   preferredhemisphere  6 non-null      object        
 4   startdate            6 non-null      datetime64[ns]
 5   enddate              6 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 416.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        366 non-null    datetime64[ns]
 1   percentage  366 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB
<class 'pandas.

Criando uma função que use uma cidade como parâmetro:

In [26]:
def predict_best_meteor_shower_viewing(city):

    # criar uma string vazia para retornar a mensagem de resposta para o usuário:
    meteor_shower_string = ""

    if city not in cities.values:
        meteor_shower_string = "Unfortunately, " + city + " isn't available for a prediction at this time."
        return meteor_shower_string

    # atribui à variável o retorno da latitude da cidade do DataFrame cities 
    latitude = cities.loc[cities['city'] == city, 'latitude'].iloc[0]   # valor da linha 0    

    # retorne a lista de constelações que são visíveis da latitude indicada:
    constellation_list = constellations.loc[(constellations['latitudestart'] >= latitude) & (constellations['latitudeend'] <= latitude), 'constellation'].tolist()  # converte em uma lista

    # avisando o usuário se nenhuma constelação for visível
    if not constellation_list:
        meteor_shower_string = "Unfortunately, there are no meteor showers viewable from "+ city + "."

        return meteor_shower_string
    
    meteor_shower_string = "In " + city + " you can see the following meteor showers:\n"  # retorno

    # loop para verificar cada constelação que é visível em cada cidade
    for constellation in constellation_list:

        # encontre a chuva de meteoro que é a mais próxima da constelação 
        meteor_shower = meteor_showers.loc[meteor_showers['radiant'] == constellation, 'name'].iloc[0]

        # encontre as datas iniciais e finais da chuva de meteoro
        meteor_shower_startdate = meteor_showers.loc[meteor_showers['radiant'] == constellation, 'startdate'].iloc[0]
        meteor_shower_enddate = meteor_showers.loc[meteor_showers['radiant'] == constellation, 'enddate'].iloc[0]

        # encontre as fases da lua para cada fase dentro do frame de tempo de visualização da chuva de meteoros
        moon_phases_list = moon_phases.loc[(moon_phases['date'] >= meteor_shower_startdate) & (moon_phases['date'] <= meteor_shower_enddate)]

        # encontrar a primeira data em que a lua está menos visível
        best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmin()]['date']

        # incluir os dados do filme
        if meteor_shower == 'Chang\'e':

            # encontrar a data em que a lua está mais visível
            best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmax()]['date']

            # adicionar a data ao string para retornar ao usuário
            meteor_shower_string += "Though the Moon will be bright, " + meteor_shower + "'s meteor shower is best seen if you look towards the " + constellation + " constellation on " +  best_moon_date.to_pydatetime().strftime("%B %d, %Y") + ".\n"
        
        else:

            # encontre a primeira data em que a lua está menos visível
            best_moon_date = moon_phases_list.loc[moon_phases_list['percentage'].idxmin()]['date']

            # adicionar a data ao string para retornar ao usuário
            meteor_shower_string += meteor_shower + " is best seen if you look towards the " + constellation + " constellation on " +  best_moon_date.to_pydatetime().strftime("%B %d, %Y") + ".\n"
    
    return meteor_shower_string

# Parte 3 - Adicionando dados da história de Chang'e

Foram criadas entradas da chuva de meteoro Chang'e e da constelação Draco para incluir os dados no DataFrame. Ambas foram colocadas respectivamente nas células 3 e 4.

In [28]:
print(predict_best_meteor_shower_viewing('Beijing'))

In Beijing you can see the following meteor showers:
Lyrids is best seen if you look towards the Lyra constellation on April 22, 2020.
Eta Aquarids is best seen if you look towards the Aquarius constellation on April 22, 2020.
Orionids is best seen if you look towards the Orion constellation on October 16, 2020.
Perseids is best seen if you look towards the Perseus constellation on July 20, 2020.
Though the Moon will be bright, Chang'e's meteor shower is best seen if you look towards the Draco constellation on October 01, 2020.



Foram incluída alterações na função preditiva para alinhá-la com o filme.